In [12]:
# Load Outages Data
import pandas as pd
ikeja_data = pd.read_csv('IKEDC.csv')
#ikeja_data = ikeja_data.drop(columns=['Unnamed: 7'])
#ikeja_data['Date'] = pd.to_datetime(ikeja_data['Date'], dayfirst=True)
#ikeja_data['Date'] = ikeja_data['date'].dt.strftime('%d-%m-%Y')
#ikeja_data['Date'] = pd.to_datetime(ikeja_data['Date'],format='%m/%d/%y')

In [13]:
ikeja_data['Date'] = pd.to_datetime(ikeja_data['Date'], format='%d/%m/%Y')

In [7]:
ikeja_data.head()

,Date,Branch,Location,Feeder,Category,Fault,Areas Affected
0,2021-10-02,Shomolu,Oworo,33 KV,Feeder,FAULT ON OWORO IGBOBI 33KV,Oworo Igbobi
1,2021-10-02,Shomolu,Oworo,33 KV,Feeder,FAULT ON OWORO IGBOBI 33KV,Oworo Igbobi
2,2021-10-04,Abule Egba,Fagba,Not Specified,Transformer (N/S),FAULT ON 11-IJAIYE OJOKOROINJ-T1-AGBADO 2-MAJE...,11-IJAIYE OJOKOROINJ-T1-AGBADO 2-MAJEOBAJE
3,2021-10-10,Abule Egba,Fagba,11 KV,Feeder,FAULT ON AGBE ROAD 11KV FEEDER,Agbe Road
4,2021-10-10,Abule Egba,Fagba,11 KV,Feeder,FAULT ON AGBE ROAD 11KV FEEDER,Agbe Road


In [15]:
ikeja_data['Location'] = ikeja_data['Location'].str.upper()
ikeja_data['Areas Affected'] = ikeja_data['Areas Affected'].str.upper()
ikeja_data['Areas Affected'] = ikeja_data['Areas Affected'].str.replace('-', ' ')

In [16]:
ikeja_data

,Date,Branch,Location,Feeder,Category,Fault,Areas Affected
0,2021-10-02,Shomolu,OWORO,33 KV,Feeder,FAULT ON OWORO IGBOBI 33KV,OWORO IGBOBI
1,2021-10-02,Shomolu,OWORO,33 KV,Feeder,FAULT ON OWORO IGBOBI 33KV,OWORO IGBOBI
2,2021-10-04,Abule Egba,FAGBA,Not Specified,Transformer (N/S),FAULT ON 11-IJAIYE OJOKOROINJ-T1-AGBADO 2-MAJE...,11 IJAIYE OJOKOROINJ T1 AGBADO 2 MAJEOBAJE
3,2021-10-10,Abule Egba,FAGBA,11 KV,Feeder,FAULT ON AGBE ROAD 11KV FEEDER,AGBE ROAD
4,2021-10-10,Abule Egba,FAGBA,11 KV,Feeder,FAULT ON AGBE ROAD 11KV FEEDER,AGBE ROAD
...,...,...,...,...,...,...,...
553,2023-11-27,Ikeja,OREGUN,Not Specified,Feeder,OUTAGE ON 11-OPEBIINJ-T1-SALVATION,11 OPEBIINJ T1 SALVATION
554,2023-11-28,Abule Egba,ADIYAN,Not Specified,Notification,REQUEST TO NOTIFY CUSTOMERS ON MATOGUN 11KV FE...,MATOGUN
555,2023-12-02,Oshodi,IGANDO,33 KV,Feeder,OUTAGE ON IGANDO 33KV FEEDER,"11 IGANDOINJ T2 GENERAL HOSPITAL ODUNBAYO, 11 ..."
556,2023-12-02,Akowonjo,ABULE,Not Specified,Substation,CNN TO ALL CUSTOMERS FEEDING FROM ALIMOSHO 3X1...,11 ALIMOSHOINJ T4 AKOWONJO 11 ALIMOSHOINJ T4 F...


In [17]:
# Define the date range
start_date = min(ikeja_data['Date'])
end_date = max(ikeja_data['Date'])
date_range = pd.date_range(start_date, end_date)

# Create a DataFrame with the date range
all_dates_df = pd.DataFrame({'Date': date_range})

# Merge the existing DataFrame with the date range DataFrame
merged_df = pd.merge(all_dates_df, ikeja_data, on='Date', how='left')

# Fill missing 'Fault' values with "None"
merged_df['Fault'].fillna('None', inplace=True)

In [18]:
neso_data = pd.read_csv('neso/neso_power_data.csv')

In [19]:
merged_df['Date'] = pd.to_datetime(merged_df['Date'])
neso_data['Date'] = pd.to_datetime(neso_data['Date'])

# Merge the data based on the 'Date' column
merged_data = pd.merge(merged_df, neso_data, on='Date', how='left')

In [47]:
!pwd

/Users/mac/Downloads/project


In [20]:
import os
weather_data_dir = 'weather_data/'

# List of branch names (derived from file names)
branch_names = [file.split('_weather.csv')[0] for file in os.listdir(weather_data_dir) if file.endswith('_weather.csv')]

In [21]:
# Merge the data for each branch
for branch_name in branch_names:
    # Read the weather data for the current branch
    branch_weather_data = pd.read_csv(os.path.join(weather_data_dir, f'{branch_name}_weather.csv'), parse_dates=['datetime'])
    
    # Merge on 'datetime' and 'Branch'
    complete_data = pd.merge(merged_data, branch_weather_data, left_on=['Date'], right_on=['datetime'], how='left', suffixes=('', f'_{branch_name}'))

In [23]:
complete_data.to_csv('complete_data.csv', index=False)

In [22]:
complete_data.head(4)

,Date,Branch,Location,Feeder,Category,Fault,Areas Affected,Peak Generation,Daily Energy Generation,Lowest Energy Generation,...,solarenergy,uvindex,severerisk,sunrise,sunset,moonphase,conditions,description,icon,stations
0,2021-10-02,Shomolu,OWORO,33 KV,Feeder,FAULT ON OWORO IGBOBI 33KV,OWORO IGBOBI,4574.23,96086.18,3321.9,...,11.2,5,NaN,2021-10-02T06:34:10,2021-10-02T18:37:26,0.87,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"DNMM,65201099999,VRJT5"
1,2021-10-02,Shomolu,OWORO,33 KV,Feeder,FAULT ON OWORO IGBOBI 33KV,OWORO IGBOBI,4574.23,96086.18,3321.9,...,11.2,5,NaN,2021-10-02T06:34:10,2021-10-02T18:37:26,0.87,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"DNMM,65201099999,VRJT5"
2,2021-10-03,NaN,NaN,NaN,NaN,None,NaN,4849.03,95387.46,3284.0,...,11.2,4,NaN,2021-10-03T06:34:01,2021-10-03T18:36:56,0.90,"Rain, Partially cloudy",Partly cloudy throughout the day with rain.,rain,"DNMM,remote,65201099999,VRJT5"
3,2021-10-04,Abule Egba,FAGBA,Not Specified,Transformer (N/S),FAULT ON 11-IJAIYE OJOKOROINJ-T1-AGBADO 2-MAJE...,11 IJAIYE OJOKOROINJ T1 AGBADO 2 MAJEOBAJE,4932.41,103650.50,3473.8,...,19.1,8,NaN,2021-10-04T06:33:53,2021-10-04T18:36:27,0.94,"Rain, Partially cloudy",Partly cloudy throughout the day with a chance...,rain,"DNMM,remote,VRJT5,65201099999"
